In [1]:
import numpy as np
from sklearn.model_selection import KFold
from sklearn import datasets
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import pandas as pd

In [2]:
train=pd.read_csv('train.csv')
test=pd.read_csv('test.csv')
train


,distance_from_home,distance_from_last_transaction,ratio_to_median_purchase_price,repeat_retailer,used_chip,used_pin_number,online_order,fraud
0,2.131956,56.372401,6.358667,1,0,0,1,1
1,3.803057,67.241081,1.872950,1,0,0,1,1
2,15.694986,175.989182,0.855623,1,0,0,1,1
3,26.711462,1.552008,4.603601,1,1,0,1,1
4,10.664474,1.565769,4.886521,1,0,0,1,1
...,...,...,...,...,...,...,...,...
19995,2.895796,0.625744,2.734760,1,1,0,1,0
19996,7.386646,225.860550,2.019975,1,0,1,1,0
19997,2.977114,1.097265,0.429777,1,1,0,0,0
19998,13.794286,0.309331,1.838496,1,0,0,1,0


In [5]:
train.isnull().sum()

distance_from_home                0
distance_from_last_transaction    0
ratio_to_median_purchase_price    0
repeat_retailer                   0
used_chip                         0
used_pin_number                   0
online_order                      0
fraud                             0
dtype: int64

In [14]:

x_train=train.iloc[:,:-1].values
y_train=train.iloc[:,-1].values
x_test=test.values

In [13]:
#归一化
from sklearn.preprocessing import StandardScaler,MinMaxScaler
# scaler = StandardScaler()
scaler = MinMaxScaler(feature_range=(0, 1))
scaler.fit(x_train)
x_train_scaler = scaler.transform(x_train)
x_test_scaler = scaler.transform(x_test)

In [6]:
#从训练数据划分验证集
#重新划分划分测试集和训练集
from sklearn.model_selection import train_test_split
x_train=x_train_scaler
# y_train=y_train
# x_train, x_valid, y_train, y_valid = train_test_split(x_train_scaler, y_train, test_size=0.2, random_state=2023)

In [7]:
class MyStacking:

    # 初始化模型参数
    def __init__(self, estimators, final_estimator, cv=5, method='predict'):
        self.cv = cv
        self.method = method
        self.estimators = estimators
        self.final_estimator = final_estimator

    # 模型训练
    def fit(self, X, y):
        # 获得一级输出
        dataset_train = self.stacking(X, y)

        # 模型融合
        self.final_estimator.fit(dataset_train, y)

    # 堆叠输出
    def stacking(self, X, y):
        kf = KFold(n_splits=self.cv, shuffle=True, random_state=2021)
        # 获得一级输出
        dataset_train = np.zeros((X.shape[0], len(self.estimators)))
        for i, model in enumerate(self.estimators):
            for (train, val) in kf.split(X, y):
                X_train = X[train]
                X_val = X[val]
                y_train = y[train]

                y_val_pred = model.fit(X_train, y_train).predict(X_val)
                dataset_train[val, i] = y_val_pred
            self.estimators[i] = model
        return dataset_train

    # 模型预测
    def predict(self, X):
        datasets_test = np.zeros((X.shape[0], len(self.estimators)))
        for i, model in enumerate(self.estimators):
            datasets_test[:, i] = model.predict(X)
            print('The {} of {} finish'.format(i,model))

        return self.final_estimator.predict(datasets_test)

    # 模型精度
    def score(self, X, y):
        datasets_test = np.zeros((X.shape[0], len(self.estimators)))
        for i, model in enumerate(self.estimators):
            datasets_test[:, i] = model.predict(X)
        return self.final_estimator.score(datasets_test, y)

In [8]:
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
estimators = [SVC(random_state=10),
              RandomForestClassifier(random_state=10),
              LogisticRegression(random_state=10),
              DecisionTreeClassifier(criterion='gini',random_state=0)]

In [9]:
clf = MyStacking(estimators=estimators,final_estimator=GradientBoostingClassifier(n_estimators=20))

In [ ]:
clf.fit(x_train, y_train)

In [ ]:
y_pred=clf.predict(x_test_scaler)

In [ ]:
y_pred

In [ ]:
out=pd.DataFrame(y_pred)
out

In [ ]:
out.to_csv('./pred.csv',index=False,header=False)